# Part 2: Basic queries - solutions

### Goals
* To make simple requests to SRU
* To display content of a record

### How to use SRU
#### SRU servers
* https://swisscovery.slsp.ch/view/sru/41SLSP_NETWORK?version=1.2&operation=explain
* https://renouvaud.primo.exlibrisgroup.com/view/sru/41BCULAUSA_NETWORK?version=1.2&operation=explain
* https://www.sudoc.abes.fr/cbs/sru/?operation=explain&version=1.1

#### Documentation
* https://slsp.ch/fr/metadonnees/
* https://developers.exlibrisgroup.com/alma/integrations/sru/
* https://knowledge.exlibrisgroup.com/Alma/Product_Documentation/Alma_Online_Help_(Francais)/130Int%C3%A9grations_avec_des_syst%C3%A8mes_externes/030Gestion_des_ressources/170Recherche_SRU%2F%2FSRW
* https://abes.fr/wp-content/uploads/2023/05/guide-utilisation-service-sru-catalogue-sudoc.pdf

In [29]:
# Example of code

import requests
from lxml import etree

parser = etree.XMLParser(remove_blank_text=True, remove_comments=True, ns_clean=True)

ns = {
    "srw": "http://www.loc.gov/zing/srw/",
    "marc": "http://www.loc.gov/MARC21/slim"
}

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.title=arbres',
    'maximumRecords': '5'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
print(r.text)

<?xml version="1.0" encoding="UTF-8" standalone="no"?><searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
  <version>1.2</version>
  <numberOfRecords>3191</numberOfRecords>
  <records>
    <record>
      <recordSchema>marcxml</recordSchema>
      <recordPacking>xml</recordPacking>
      <recordData>
        <record xmlns="http://www.loc.gov/MARC21/slim">
          <leader>01043nam a22002892c 4500</leader>
          <controlfield tag="001">991018314729705501</controlfield>
          <controlfield tag="005">20201222201316.0</controlfield>
          <controlfield tag="008">130826r19671788xxu||||| |||| 00| ||eng d</controlfield>
          <datafield ind1=" " ind2=" " tag="035">
            <subfield code="a">(swissbib)117972746-41slsp_network</subfield>
          </datafield>
          <datafield ind1=" " ind2=" " tag="035">
            <subfield code="a">117972746</subfield>
            <subfield code="9">ExL</subfield>
          </datafield>
          <datafield ind1=" " ind2="

In [30]:
# Display 245a
xml = etree.fromstring(r.content)
f245s = xml.findall('.//marc:datafield[@tag="245"]/marc:subfield[@code="a"]', namespaces=ns)
for f245 in f245s:
    print(f245.text)

... Catalogue alphabétique des arbres et arbrisseaux
<<L>>'art de planter
<<L>>'élagage des arbres
"Au-delà du fleuve et sous les arbres" d'Ernest Hemingway
"Aux temps où arbres et plantes disaient des choses"


### Some simple requests
1. Find 10 documents of Slatkine editions
2. Build a function to display titles in the folowing way: `<245a> (<100a>), <264c>` (you can choose other format), display Slatkine titles
3. Find a document of 2020 with python in the title
4. Try other SRU servers

In [31]:
# 1. Find 10 documents of Slatkine editions
# You need to find the right index in the "explain" documentation
# 
import requests

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.publisher=Slatkine',
    'maximumRecords': '10'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
print(r.text)

<?xml version="1.0" encoding="UTF-8" standalone="no"?><searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
  <version>1.2</version>
  <numberOfRecords>14094</numberOfRecords>
  <records>
    <record>
      <recordSchema>marcxml</recordSchema>
      <recordPacking>xml</recordPacking>
      <recordData>
        <record xmlns="http://www.loc.gov/MARC21/slim">
          <leader>01011nam a2200301 c 4500</leader>
          <controlfield tag="001">991157888299705501</controlfield>
          <controlfield tag="005">20230410130150.0</controlfield>
          <controlfield tag="008">130423s2013    sz ||||| |||| 00| ||fre d</controlfield>
          <datafield ind1=" " ind2=" " tag="020">
            <subfield code="a">9782832105481</subfield>
          </datafield>
          <datafield ind1=" " ind2=" " tag="035">
            <subfield code="a">(swissbib)117264172-41slsp_network</subfield>
          </datafield>
          <datafield ind1=" " ind2=" " tag="035">
            <subfield code=

In [32]:
# 2. Build a function to display titles in the folowing way: <245a> (<100a>), <260c>
# (you can choose other format), display Slatkine titles

xml = etree.fromstring(r.content)
records = xml.findall('.//marc:record', namespaces=ns)

In [33]:
records

[<Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a8362280>,
 <Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a8362840>,
 <Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a8362580>,
 <Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a8362940>,
 <Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a8362700>,
 <Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a83620c0>,
 <Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a8362e80>,
 <Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a8362740>,
 <Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a8362140>,
 <Element {http://www.loc.gov/MARC21/slim}record at 0x7fe4a8362640>]

In [34]:
def display_ref(record):
    f245a = record.findtext('.//marc:datafield[@tag="245"]/marc:subfield[@code="a"]', namespaces=ns)
    f100a = record.findtext('.//marc:datafield[@tag="100"]/marc:subfield[@code="a"]', namespaces=ns)
    f260c = record.findtext('.//marc:datafield[@tag="264"]/marc:subfield[@code="c"]', namespaces=ns)
    return f'{f245a} ({f100a}), {f260c}'

In [35]:
for record in records:
    print(display_ref(record))

... Et dans le brouillard (Brugger, Mélanie), 2013
<<L>>'aggettivazione nei "Contes" di Voltaire dall'"Histoire d'un bon Bramin" alla "Princesse de Babylon" (Giacomelli Deslex, Marcella), 1984
'Ann' Quin bredouille, ou, Le petit cousin de Tristam Shandy (1791-1792) (Gorjy, Jean-Claude), 2012
<<L>>'énigme Jaccoud (Jaquet, Corinne), 2020
<<L>>'idée du peintre parfait pour servir de regle aux jugements que l'on doit porter sur les ouvrages des peintres (None), 1970
<<Le>> 'tempeste della vita' (Tatti, Mariasilvia), 1999
"<<Le>> fou d'Elsa" de Louis Aragon (Burle-Errecade, Élodie), 2023
"A l'ombre des jeunes filles en fleurs" de Marcel Proust (Rey, Pierre-Louis), 1983
"A l'ombre des jeunes filles en fleurs" de Marcel Proust (Rey, Pierre-Louis), 1983
"A la recherche du temps perdu" en France et en Allemagne (Fravalo-Tane, Pascale), 2008


In [41]:
# 3. Find a document of 2020 with python in the title

import requests

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.title=python and alma.main_pub_date=2020',
    'maximumRecords': '1'
}

url = 'https://swisscovery.ch/view/sru/41SLSP_NETWORK'

r = requests.get(url, params=params)
print(r.text)

<?xml version="1.0" encoding="UTF-8" standalone="no"?><searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
  <version>1.2</version>
  <numberOfRecords>476</numberOfRecords>
  <records>
    <record>
      <recordSchema>marcxml</recordSchema>
      <recordPacking>xml</recordPacking>
      <recordData>
        <record xmlns="http://www.loc.gov/MARC21/slim">
          <leader>00931nam  2200277 i 4500</leader>
          <controlfield tag="001">991170624062705501</controlfield>
          <controlfield tag="005">20230822232842.0</controlfield>
          <controlfield tag="006">m     o  d |</controlfield>
          <controlfield tag="007">cr cnu||||||||</controlfield>
          <controlfield tag="008">200421s2020    xx      o     000 0 eng d</controlfield>
          <datafield ind1=" " ind2=" " tag="020">
            <subfield code="a">0-9984014-2-0</subfield>
          </datafield>
          <datafield ind1=" " ind2=" " tag="035">
            <subfield code="a">(CKB)4100000010564620<

In [42]:
# 4. Find a document of 2020 with python in the title in another SRU server

import requests

params = {
    'version': '1.2',
    'operation': 'searchRetrieve',
    'query': 'alma.title=python and alma.main_pub_date=2020',
    'maximumRecords': '1'
}

url = 'https://renouvaud.primo.exlibrisgroup.com/view/sru/41BCULAUSA_NETWORK'

r = requests.get(url, params=params)
print(r.text)

<?xml version="1.0" encoding="UTF-8" standalone="no"?><searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/">
  <version>1.2</version>
  <numberOfRecords>95</numberOfRecords>
  <records>
    <record>
      <recordSchema>marcxml</recordSchema>
      <recordPacking>xml</recordPacking>
      <recordData>
        <record xmlns="http://www.loc.gov/MARC21/slim">
          <leader>00840naa a2200229 u 4500</leader>
          <controlfield tag="001">991023754315202851</controlfield>
          <controlfield tag="005">20250420095309.0</controlfield>
          <controlfield tag="008">210222s2020    sz |||||||||||0|||0|fre d</controlfield>
          <datafield ind1=" " ind2=" " tag="035">
            <subfield code="a">(RNV_B)0002607932</subfield>
          </datafield>
          <datafield ind1=" " ind2="7" tag="039">
            <subfield code="b">7260</subfield>
          </datafield>
          <datafield ind1=" " ind2=" " tag="040">
            <subfield code="a">RNV vdcite</subfield>
   